## finetuning de BETO con el dataset etiquetado original

In [1]:
#!pip install datasets seaborn



In [1]:
# no lograba bajar el dataset porque
# me fallaba bajar archivos de hugginface y tenia que actualizar esto (JUAN MA me ayudo)
#pip install -U datasets

In [2]:
from datasets import load_dataset
import pandas as pd
from datetime import datetime, timedelta
from tqdm.auto import tqdm
from collections import Counter

(1) Dataset de etiquetado

https://huggingface.co/datasets/piuba-bigdata/contextualized_hate_speech

Este es el dataset original de etiquetado. Contiene los siguientes campos:

ID : identificador 

TITLE: título de la noticia 

TEXT: texto del tweet del comentario al Tweet original

CONTEXT: tweet original

HATEFUL: 1 hateful, 0 not hateful

BODY: texto de la noticia

CALLS: si se realizó o no un llamado a la acción

WOMEN: discriminación hacia mujeres

LGBTI: discriminación hacia grupos LGBTI

RACISM: discriminación asociada a racismo

CLASS: discriminación vinculada a pobreza, villas

POLITICS: odio o violencia vinculada a las ideas políticas

DISABLED: discriminación a las personas con discapacidad

APPEARENCE: discriminación en base al aspecto físico (incluye especialmente la gordofobia)

CRIMINAL: discriminación hacia personas en el sistema penal o similares



In [4]:
# para ver las versiones de las librerias
#pip freeze

In [3]:
from datasets import load_dataset

ds1 = load_dataset("piuba-bigdata/contextualized_hate_speech")



In [4]:
ds1

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 36420
    })
    test: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 11343
    })
    dev: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 9106
    })
})

In [7]:
##Esto era para levantarlo desde parquet cuando no pude hacerlo con HUGGINFACE
## cargo  train, test y dev por separado
##ds1_train=pd.read_parquet('train/train-00000-of-00001.parquet', engine='pyarrow')
###ds1_train

El dataset 1 contiene 56869 tw etiquetados.
El dataset se dividió en 36420 de train, test:11343 y vaLidacion:9106

In [5]:
ds1_train=ds1["train"]
ds1_train

Dataset({
    features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
    num_rows: 36420
})

In [6]:
example1= ds1_train[30355]
example1

{'id': 374451,
 'title': 'Asalto y tiroteo en un colectivo en Villa Lugano: murió un ladrón de 15 años',
 'text': '@usuario Uno menos.',
 'context_tweet': 'Asalto y tiroteo en un colectivo en Villa Lugano: murió un ladrón de 15 años https://t.co/arXOD0w5xp',
 'HATEFUL': 1,
 'body': 'Dos delincuentes subieron a un colectivo de la línea 47 en el barrio porteño de Villa Lugano, muy cerca de la General Paz y autopista Riccheri, con intenciones de asaltar a todos pero se encontraron con que entre los pasajeros viajaba una mujer policía vestida de civil que resistió el robo a los tiros.\n\nSegún fuentes oficiales, el hecho ocurrió durante la madrugada en el interno 106 de la línea 47. La mujer policía que estaba franco de servicio defendió a los pasajeros y al chofer y se enfrentó a los delincuentes.\n\nTodo comenzó cuando en la parada ubicada en Albariño y Riestra, subieron dos personas armadas. Los testigos comentaron que mientras uno le robaba el celular a una mujer y el otro amenazaba co

## Aplicando distintos modelos para entrenar. Usando el siguient tutorial
Uso  de base el siguiente tutorial para hacer el finetuning:
https://huggingface.co/blog/sentiment-analysis-python
y este:
https://huggingface.co/docs/transformers/training


La idea es utilizar la librería tranbsformers de huggingface para hacerle finetuning a un BETO original, es decir la version de BERT pre-entrenada con español de GOOGLE pero que no haya sido reentrenado con ningún otro dataset.
Aqui para usar transformers de hugginface
https://github.com/huggingface/transformers?tab=readme-ov-file



### 1. pre-process data

In [7]:
 ds1['train'][0]

{'id': 343726,
 'title': 'Video: salió de la cárcel por el coronavirus y murió de un tiro el mismo día al festejar su libertad',
 'text': '@usuario Uno menos',
 'context_tweet': 'Video: salió de la cárcel por el coronavirus y murió de un tiro el mismo día al festejar su libertad https://t.co/UrJGRAdEvD',
 'HATEFUL': 1,
 'body': 'Un hombre de 46 años, que cumplía una condena en prisión y fue puesto en libertad por el coronavirus, murió el mismo día que dejó la cárcel baleado por un primo, durante los "festejos" por dejar la prisión.\n\nSari Salem Wardat, de 46 años, recibió un disparo desde atrás en un aparente accidente mientras caminaba hacia miembros de la familia y vecinos que se habían reunido para darle la bienvenida a su casa, en Jordania.\n\nUn video que circuló en las redes sociales \u200bmuestra a Sari llegando a su casa en un auto azul rodeado de tiros de "celebración". El primo se acerca de inmediato a Sari, que sostiene una pistola y la apunta hacia el cielo.\n\nEl detenido

In [10]:
ds_train_small= ds1['train'].select_columns(["text","HATEFUL"])
ds_test_small= ds1['test'].select_columns(["text","HATEFUL"])

In [11]:
##aca les coloco el mismo nombre que espera el modelo (me parece cualquiera pero no logre que ande sino)
ds_train_small=ds_train_small.rename_column("HATEFUL", "label")
ds_test_small=ds_test_small.rename_column("HATEFUL", "label")

In [12]:
ds_train_small

Dataset({
    features: ['text', 'label'],
    num_rows: 36420
})


Uso esta instancia de BETO porque no encontré una original.
https://github.com/dccuchile/beto?tab=readme-ov-file

Es decir, este: dccuchile/bert-base-spanish-wwm-uncased


In [13]:
import torch
from transformers import BertForMaskedLM, BertTokenizer

Utiizo las librerías de transformers para preparar los daos de entrenamiento

Lo primero es tokenizar en función del modelo seleccionado

In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

tokenizer_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

Le aplico la función tokenizer del modelo BETO a mis dos conjuntos, el de entrenamiento y el de test.

In [15]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)
 
tokenized_train = ds_train_small.map(preprocess_function, batched=True)
tokenized_test = ds_test_small.map(preprocess_function, batched=True)

Map:   0%|          | 0/36420 [00:00<?, ? examples/s]

Map:   0%|          | 0/11343 [00:00<?, ? examples/s]

In [16]:
tokenized_train[0]

{'text': '@usuario Uno menos',
 'label': 1,
 'input_ids': [4, 985, 8563, 1614, 1885, 5],
 'token_type_ids': [0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1]}

La siguiente función (según el ejmplo) transforma los datos de entrenamiento agregandole el padding 

Se usará luego en el Trainer

In [20]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## 2 Entreno el modelo. 

Voy a entrenar el modelo con el TRAINER de transformers. Instancio el modelo pre-entrenado BETO y aca selecciono 2 labels 

dccuchile/bert-base-spanish-wwm-uncased


AGUS : ACA TENDRIA QUE CAMBIAR LA CANTIDAD DE LABELS PARA PODER ENTRENAR CON TODAS LAS ETIQUETAS?

In [17]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", num_labels=2)


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuc

In [18]:
##FUNCION PARA COMPUTAR LAS METRIcas

import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [21]:
from transformers import TrainingArguments, Trainer
 
#repo_name = "finetuning-sentiment-model-3000-samples"
 
training_args = TrainingArguments(
    output_dir="output-beto1",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    seed=0,
    load_best_model_at_end=True,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [22]:
# Train pre-trained model
trainer.train()

C:\Users\Nat\AppData\Roaming\Python\Python311\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/13659 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.4316, 'learning_rate': 4.816970495643898e-05, 'epoch': 0.11}


  0%|          | 0/1418 [00:00<?, ?it/s]

...

  0%|          | 0/1418 [00:00<?, ?it/s]

C:\Users\Nat\AppData\Roaming\Python\Python311\site-packages\datasets\load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
C:\Users\Nat\AppData\Roaming\Python\Python311\site-packages\datasets\load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandator

{'eval_loss': 0.45886319875717163, 'eval_accuracy': 0.8415763025654589, 'eval_f1': 0.0, 'eval_runtime': 36427.8575, 'eval_samples_per_second': 0.311, 'eval_steps_per_second': 0.039, 'epoch': 0.55}
{'loss': 0.4528, 'learning_rate': 3.901822973863387e-05, 'epoch': 0.66}


  0%|          | 0/1418 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
output

In [ ]:
predictions = output.logits.argmax(dim=-1)
predictions

PRUEBO EL MODELO DE PIUBAMAS CON EL CODIGO DE JM

In [ ]:
##codigo de JM de Huggingface
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "piubamas/beto-contextualized-hate-speech"
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

id2label = [model.config.id2label[k] for k in range(len(model.config.id2label))]

def predict(*args):
    encoding = tokenizer.encode_plus(*args)

    inputs = {
        k:torch.LongTensor(encoding[k]).reshape(1, -1) for k in {"input_ids", "attention_mask", "token_type_ids"}
    }

    output = model.forward(
        **inputs
    )

    chars = list(zip(id2label, list(output.logits[0].detach().cpu().numpy() > 0)))

    return [char for char, pred in chars if pred]



In [ ]:
#df_tweets

In [ ]:
context = "China prohíbe la cría de perros para consumo humano"
text = "Chinos hdrmp hay que matarlos a todos"

prediction = predict(text, context)

In [ ]:
prediction